In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
sns.set_theme(style="whitegrid")

data = pd.read_csv('https://raw.githubusercontent.com/Fluteman84/TechIntern/main/Sportcarprice.csv')

In [3]:
data


,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
0,Porsche,911,2022,3,379,331,4,"101,200"
1,Lamborghini,Huracan,2021,5.2,630,443,2.8,"274,390"
2,Ferrari,488 GTB,2022,3.9,661,561,3,"333,750"
3,Audi,R8,2022,5.2,562,406,3.2,"142,700"
4,McLaren,720S,2021,4,710,568,2.7,"298,000"
...,...,...,...,...,...,...,...,...
1002,Koenigsegg,Jesko,2022,5,1280,1106,2.5,"3,000,000"
1003,Lotus,Evija,2021,Electric Motor,1972,1254,2,"2,000,000"
1004,McLaren,Senna,2021,4,789,590,2.7,"1,000,000"
1005,Pagani,Huayra,2021,6,764,738,3,"2,600,000"


In [4]:
data.isnull().sum()

Car Make                    0
Car Model                   0
Year                        0
Engine Size (L)            10
Horsepower                  0
Torque (lb-ft)              3
0-60 MPH Time (seconds)     0
Price (in USD)              0
dtype: int64

In [5]:
data = data.drop(columns = '0-60 MPH Time (seconds)',axis =1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Car Make         1007 non-null   object
 1   Car Model        1007 non-null   object
 2   Year             1007 non-null   int64 
 3   Engine Size (L)  997 non-null    object
 4   Horsepower       1007 non-null   object
 5   Torque (lb-ft)   1004 non-null   object
 6   Price (in USD)   1007 non-null   object
dtypes: int64(1), object(6)
memory usage: 55.2+ KB


In [7]:
data['Price (in USD)'] = data['Price (in USD)'].str.replace(',','')

In [8]:
data['Price (in USD)'] = data['Price (in USD)'].astype(int)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Car Make         1007 non-null   object
 1   Car Model        1007 non-null   object
 2   Year             1007 non-null   int64 
 3   Engine Size (L)  997 non-null    object
 4   Horsepower       1007 non-null   object
 5   Torque (lb-ft)   1004 non-null   object
 6   Price (in USD)   1007 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 55.2+ KB


In [11]:
data  = data.drop(columns = 'Torque (lb-ft)',axis=1)

In [13]:
data.isnull().sum()

Car Make            0
Car Model           0
Year                0
Engine Size (L)    10
Horsepower          0
Price (in USD)      0
dtype: int64

In [14]:
data = data.reset_index(drop = True)

In [16]:
data.describe()

,Year,Price (in USD)
count,1007.000000,1.007000e+03
mean,2021.201589,3.820359e+05
std,2.019802,7.383227e+05
min,1965.000000,2.500000e+04
25%,2021.000000,7.180000e+04
50%,2021.000000,1.400000e+05
75%,2022.000000,2.500000e+05
max,2023.000000,5.200000e+06


In [17]:
data[data['Price (in USD)']>5e6]

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Price (in USD)
541,Bugatti,Chiron Super Sport 300+,2022,8,1578,5200000
823,Bugatti,Chiron Super Sport 300+,2021,8,1578,5200000


In [18]:
data = data[data['Price (in USD)']<5e6].reset_index(drop=True)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Car Make         1005 non-null   object
 1   Car Model        1005 non-null   object
 2   Year             1005 non-null   int64 
 3   Engine Size (L)  995 non-null    object
 4   Horsepower       1005 non-null   object
 5   Price (in USD)   1005 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 47.2+ KB


In [20]:
 data = data[~data['Horsepower'].isna()]

In [21]:
x=data.drop(columns='Price (in USD)',axis=1)
y=data['Price (in USD)']

In [22]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = 0.2)

In [23]:
ohe = OneHotEncoder()

In [26]:
ohe.categories

'auto'

In [27]:
ohe.fit(x[['Car Make','Car Model','Engine Size (L)','Horsepower']])

OneHotEncoder()

In [28]:
column_trans = make_column_transformer((OneHotEncoder(categories = ohe.categories_),['Car Make','Car Model','Engine Size (L)','Horsepower']),remainder='passthrough')

In [29]:
lr = LinearRegression()

In [30]:
pipe=make_pipeline(column_trans,lr)

In [31]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Acura', 'Alfa Romeo', 'Alpine', 'Ariel', 'Aston Martin', 'Audi',
       'BMW', 'Bentley', 'Bugatti', 'Chevrolet', 'Dodge', 'Ferrari',
       'Ford', 'Jaguar', 'Kia', 'Koenigsegg', 'Lamborghini', 'Lexus',
       'Lotus', 'Maserati', 'Mazda', 'McLaren'...
       '591', '592', '600', '603', '611', '612', '616', '617', '620',
       '621', '622', '624', '625', '626', '630', '640', '641', '645',
       '647', '650', '660', '661', '671', '689', '690', '707', '710',
       '715', '717', '720', '729', '730', '750', '755', '759', '760',
       '764', '770', '780', '789', '791', '797', '800', '819', '887',
       '986'], dtype=object)]),
                                                  ['Car Make', 'Car Model',
                                                   'Engine Size (L)',
                                                   'Horsepower'])])),
                ('linearregression', LinearRegression())])

In [32]:
y_pred = pipe.predict(x_test)

In [34]:
r2_score(y_test,y_pred)

0.8614676549358974

In [35]:
scores=[]
for i in range(10):
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = i)
    lr = LinearRegression()
    pipe = make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    scores.append(r2_score(y_test,y_pred))

In [36]:
print(r2_score(y_test,y_pred),i)

0.9128367394212257 9


In [37]:
np.argmax(scores)

1

In [38]:
scores[np.argmax(scores)]

0.9920392801147885

In [40]:
r2_score(y_test,y_pred)

0.9920392801147885